# Knot Hash

## Part 1

In [128]:
import numpy as np

def invert(a, length, head):
    r = np.roll(a, -head)
    r = np.concatenate((r[:length][::-1], r[length:]))
    return np.roll(r, head)

def transform(a, length, head, skip, size=2**8):
    r = invert(a, length, head)
    head = (head + length + skip) % size
    return r, head

def one_round(input_lengths, a=None, skip=0, head=0, size=2**8):
    if a is None:
        a = np.arange(size)
    for length in input_lengths:
        a, head = transform(a, length, head, skip, size)
        skip += 1
    return a, head, skip

### Test

In [129]:
a, head, skip = one_round([3, 4, 1, 5], size=5)
print(a)
print(a[0] * a[1])

[3 4 2 1 0]
12


### Solution

In [130]:
INPUT_LENGTHS = [192, 69, 168, 160, 78, 1, 166, 28, 0, 83, 198, 2, 254, 255, 41, 12]

In [131]:
a, head, skip = one_round(INPUT_LENGTHS)
print(a)
print(a[0] * a[1])

[191 255 254 253 252 251 250 249 237 238 239 240 241 242 243 244 245 246
 247 248 236 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140
 168 167 166 165 164 163 162 161 160 159 158 157 156 155 154 153 152 151
 150 149 148 147 146 145 144 143 142 141 169 170 171 172  94  93 183 184
 185  17  16  91  90  89  88  87  86  85  84  83  82  81  80  79  78  77
  76  75  74  73  72  71  70  69  68  67  66  65  64  63  62  61  60  59
  58  57  56  55  54  53  52  51  50  49  48  47  46  45  44  43  42  41
  40  39  38  37  36  35  34  33  32  31  30  29  28  27  26  25  24  23
  22  21  20  19  18 186 192 193 194 195 196 197 198 199 200 201 202 203
 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221
 222 223 224 225 226   9  10  11  12  13  14  15  92 182 181 179 180 178
 177 176 175 174 173  95  96  97  99  98 100   2   3   4   5   6   7   8
 227 228 229 230 231 232 233 234 235 125 124 123 122 121 120 119 118 117
 116 115 114 113 112 111 110 109 108 107 106 105 10

## Part 2

In [132]:
standard_suffix = [17, 31, 73, 47, 23]

def lengths_from_string(ascii_input):
    return list(map(ord, list(ascii_input))) + standard_suffix

In [133]:
def many_rounds(input_lengths, nruns = 2**6):
    head, skip = 0, 0
    a = np.arange(2**8)
    for _ in range(nruns):
        a, head, skip = one_round(input_lengths, a=a, skip=skip, head=head)
    return a

In [134]:
from functools import reduce
import operator

def dense(h):
    dense_hash = []
    for i in range(2**4):
        a = h[16 * i: 16 * (i + 1)]
        dense_hash.append(reduce(operator.xor, a, 0))
    return dense_hash

In [135]:
def hashing(ascii_input, size=2**8):
    lengths = lengths_from_string(ascii_input)
    dense_hash = dense(many_rounds(lengths))
    return ''.join([hex(d)[2:] if len(hex(d)) == 4 else '0' + hex(d)[2:] for d in dense_hash])

### Test

In [136]:
def test():
    assert(hashing('') == 'a2582a3a0e66e6e86e3812dcb672a272')
    assert(hashing('AoC 2017') == '33efeb34ea91902bb2f59c9920caa6cd')
    assert(hashing('1,2,3') == '3efbe78a8d82f29979031a4aa0b16a9d')
    assert(hashing('1,2,4') == '63960835bcdc130f0b66d7ff4f6a5a8e')
test()

### Solution

In [137]:
ascii_input = ','.join(list(map(str, INPUT_LENGTHS)))
hashing(ascii_input)

'1c46642b6f2bc21db2a2149d0aeeae5d'